<h1>Direct Stiffness Method: Beam Elements</h1>
<h3>Structural Mechanics, Boston University</h3>
<h4>Prof. Douglas P. Holmes</h4>

In [1]:
# Here are some common libraries that expand the functionality of python
import math         # sine, cosine, sqrt, etc.
import numpy as np  # so we can work with matrices

import matplotlib
import matplotlib.pyplot as plt # plotting

import sympy as sp        # symbolic calculations

<h2>Deriving the Governing Equations for a Beam Element</h2>
We will use SymPy

In [2]:
x, a1, a2, a3, a4, L = sp.symbols('x a1 a2 a3 a4 L') # deflection w(x) = a1+a2*x+a3*x^2+a4*x^3
w1, w2, theta_1, theta_2 = sp.symbols('w1 w2 theta_1 theta_2') # degrees of freedom (DoF)
E, I = sp.symbols('E I') # materials and geometry
F1, M1, F2, M2 = sp.symbols('F1 M1 F2 M2') # forces (y direction) and couples
P, q = sp.symbols('P q') # force, force/length

In [3]:
def w(x):
    return a1+a2*x+a3*x**2+a4*x**3

In [4]:
w(x)

a1 + a2*x + a3*x**2 + a4*x**3

In [5]:
# note: I've defined the RHS as negative because of how SymPy solves equations. 
# You get the correct answer either way, but the L-matrix has negative signs in opposite places
bc = [
    sp.Eq(w1,-w(0)), 
    sp.Eq(theta_1,-sp.diff(w(x),x).subs(x,0)),
    sp.Eq(w2,-w(L)),
    sp.Eq(theta_2,-sp.diff(w(x),x).subs(x,L))
]

In [6]:
bc[2]

Eq(w2, -L**3*a4 - L**2*a3 - L*a2 - a1)

In [7]:
coeffs = [a1,a2,a3,a4]

In [8]:
Lmat, DoF = sp.linear_eq_to_matrix(bc,coeffs)

In [9]:
Lmat

Matrix([
[1, 0,    0,      0],
[0, 1,    0,      0],
[1, L, L**2,   L**3],
[0, 1,  2*L, 3*L**2]])

In [10]:
Lmat.inv()

Matrix([
[      1,       0,       0,       0],
[      0,       1,       0,       0],
[-3/L**2,    -2/L,  3/L**2,    -1/L],
[ 2/L**3, L**(-2), -2/L**3, L**(-2)]])

In [11]:
# Here again, the negative sign is to be consistent with the negative sign above in bc = [...]
a = Lmat.inv()*(-DoF)

In [12]:
a1, a2, a3, a4 = a[0], a[1], a[2], a[3]

In [13]:
w(x)

theta_1*x + w1 + x**3*(theta_1/L**2 + theta_2/L**2 + 2*w1/L**3 - 2*w2/L**3) + x**2*(-2*theta_1/L - theta_2/L - 3*w1/L**2 + 3*w2/L**2)

In [14]:
forcesCouples = [
    sp.Eq(F1,  E*I*sp.diff(w(x),x,3).subs(x,0)),
    sp.Eq(M1, -E*I*sp.diff(w(x),x,2).subs(x,0)),
    sp.Eq(F2, -E*I*sp.diff(w(x),x,3).subs(x,L)),
    sp.Eq(M2,  E*I*sp.diff(w(x),x,2).subs(x,L))
]

In [15]:
forcesCouples[0]

Eq(F1, 6*E*I*(theta_1 + theta_2 + 2*w1/L - 2*w2/L)/L**2)

In [16]:
DoF = [w1, theta_1, w2, theta_2]
Kel, FM = sp.linear_eq_to_matrix(forcesCouples,DoF)

In [17]:
-FM

Matrix([
[F1],
[M1],
[F2],
[M2]])

In [18]:
-Kel

Matrix([
[ 12*E*I/L**3,  6*E*I/L**2, -12*E*I/L**3,  6*E*I/L**2],
[  6*E*I/L**2,     4*E*I/L,  -6*E*I/L**2,     2*E*I/L],
[-12*E*I/L**3, -6*E*I/L**2,  12*E*I/L**3, -6*E*I/L**2],
[  6*E*I/L**2,     2*E*I/L,  -6*E*I/L**2,     4*E*I/L]])

In [19]:
DoF

[w1, theta_1, w2, theta_2]

<h2>Solving Beam Deflection Problems</h2>
<h3>Geometry, Materials, and Boundary Conditions</h3>
Here, we will define the ($x$,$y$) coordinates of each node.

In [20]:
# [x1,y1], ... the (x,y) position of node 1, 2, 3...
nodes = np.array([
    [0,0],
    [1,0],
    [3,0]
])
# a list of all the nodes
node_numbers = np.array([*range(len(nodes))])

In [21]:
# element e is connected at nodes i and j
# element e: [node i, node j, Young's Modulus, Second Moment of Area] ... note: we'll calculate L later
elements = np.array([
    [1, 2, E, I],
    [2, 3, 2*E, I]
])

In [22]:
# dirichlet boundary conditions
# [node, DoF (w=1, theta=2), value]
bc = np.array([
    [1,1,0],
    [1,2,0],
    [3,1,0]
])

In [23]:
# neumann boundary conditions - e.g. forces
# [node, DoF (w=1,theta=2), magnitude of force]
forces = np.array([
    [2,1,-P]
])

In [24]:
# initializing the vectors F, u, and the matrix K
F = u = np.zeros(2*len(nodes))
K = np.zeros([2*len(nodes), 2*len(nodes)])

In [25]:
def XY(nodes, elements, i):
    return np.array([
        nodes[int(elements[i,0]-1)],
        nodes[int(elements[i,1]-1)]
    ])

In [26]:
# What nodes are not connected to the ith element?
# list_of_all_nodes+1... the +1 is to align python numbering (starts at 0) with our numbering (starts at 1)
def missingNodes(list_of_all_nodes, list_of_element_nodes): 
    return (list(set(list_of_all_nodes+1) - set(list_of_element_nodes+1))) 

In [27]:
# elemental stiffness matrix of a beam element in local coordinates
def Kbel(XY, E, I):
    x1, y1, x2, y2 = XY[0,0], XY[0,1], XY[1,0], XY[1,1]
    
    L = sp.Symbol('L', real=True, positive=True)

    K = np.array([
        [12, 6*L, -12, 6*L],
        [6*L, 4*L**2, -6*L, 2*L**2],
        [-12, -6*L, 12, -6*L],
        [6*L, 2*L**2, -6*L, 4*L**2]
    ])
    
    n = sp.sqrt((x2-x1)**2 + (y2-y1)**2)
    B = sp.refine(E*I/(n*L)**3, sp.Q.positive(L) & sp.Q.real(L)) 
    
    Kbel = B*K
                  
    return Kbel  

In [28]:
# augment the elemental stiffness matrix by adding rows & columns of zeroes at the missing nodes
def Kaug(Kel, element):
    num_DoF = 2 # 2 DoF per node 
    missing = np.array(missingNodes(node_numbers,elements[element,:2]-1))
    delete = (missing-num_DoF)+missing
    
    Kaug1 = np.insert(Kel,   delete, 0, axis=0)
    Kaug2 = np.insert(Kaug1, delete, 0, axis=0)
    Kaug3 = np.insert(Kaug2, delete, 0, axis=1)
    Kaug  = np.insert(Kaug3, delete, 0, axis=1)
    return Kaug

In [29]:
sp.Matrix(Kaug(Kbel(XY(nodes, elements, 1), elements[1,2], elements[1,3]), 1))

Matrix([
[0, 0,              0,               0,               0,               0],
[0, 0,              0,               0,               0,               0],
[0, 0,     3*E*I/L**3,  3*E*I/(2*L**2),     -3*E*I/L**3,  3*E*I/(2*L**2)],
[0, 0, 3*E*I/(2*L**2),           E*I/L, -3*E*I/(2*L**2),       E*I/(2*L)],
[0, 0,    -3*E*I/L**3, -3*E*I/(2*L**2),      3*E*I/L**3, -3*E*I/(2*L**2)],
[0, 0, 3*E*I/(2*L**2),       E*I/(2*L), -3*E*I/(2*L**2),           E*I/L]])

In [30]:
def MasterStiffness(nodes, elements, K):
# Calculate the Master Stiffness matrix:
# 1. Get the x & y coordinates of each node connected to element i
# 2. Calculate the stiffness matrix for element i
# 3. Augment the elemental stiffness matrix to the size of K
# 4. Add it to K, loop over the remaining elements
    for i in range(len(elements)):
        nodesXY = XY(nodes, elements, i)
        Kbeli  = Kbel(nodesXY, elements[i,2], elements[i,3])
        Kaugi = Kaug(Kbeli, i)
        K = np.add(K, Kaugi)    
    return K

In [31]:
K = MasterStiffness(nodes,elements,K)

In [32]:
sp.Matrix(K)

Matrix([
[ 12*E*I/L**3,  6*E*I/L**2,    -12*E*I/L**3,      6*E*I/L**2,             0.0,             0.0],
[  6*E*I/L**2,     4*E*I/L,     -6*E*I/L**2,         2*E*I/L,             0.0,             0.0],
[-12*E*I/L**3, -6*E*I/L**2,     15*E*I/L**3, -9*E*I/(2*L**2),     -3*E*I/L**3,  3*E*I/(2*L**2)],
[  6*E*I/L**2,     2*E*I/L, -9*E*I/(2*L**2),         5*E*I/L, -3*E*I/(2*L**2),       E*I/(2*L)],
[         0.0,         0.0,     -3*E*I/L**3, -3*E*I/(2*L**2),      3*E*I/L**3, -3*E*I/(2*L**2)],
[         0.0,         0.0,  3*E*I/(2*L**2),       E*I/(2*L), -3*E*I/(2*L**2),           E*I/L]])

In [33]:
# initialize vectors for the specified displacements and forces. these will be [1 x DoF]
u_specified=[]
F_specified=[0]*2*len(nodes)

# what DoF are specified?
for i in range(len(bc)):
    u_specified.append(2*(bc[i,0]-1)+bc[i,1]-1)
# assign the external forces to the corresponding DoF in F   
for i in range(len(forces)):    
    F_specified[(2*(forces[i,0]-1)+forces[i,1])-1]=forces[i,2]
    
u_unknown = np.delete(u,u_specified) # deleting the specified displacments from u
K_reduced = np.delete(K,u_specified,0) # deleting rows
K_reduced = np.delete(K_reduced,u_specified,1) # deleting columns
F_specified = np.delete(F_specified,u_specified) # deleting the forces that correspond to specified displacments

In [34]:
sp.Matrix(K_reduced)

Matrix([
[    15*E*I/L**3, -9*E*I/(2*L**2), 3*E*I/(2*L**2)],
[-9*E*I/(2*L**2),         5*E*I/L,      E*I/(2*L)],
[ 3*E*I/(2*L**2),       E*I/(2*L),          E*I/L]])

In [41]:
u_unknown = sp.Matrix(K_reduced).inv()*sp.Matrix(F_specified)

In [42]:
u_unknown

Matrix([
[-19*L**3*P/(132*E*I)],
[  -7*L**2*P/(44*E*I)],
[  13*L**2*P/(44*E*I)]])

In [43]:
# create a list of all the displacement degrees of freedom (DoF)
u_DoF=[]
for i in range(len(u)):
    u_DoF.append(i)

# get the index locations of the free degrees of freedom (DoF)    
u_free = (list(set(u_DoF) - set(u_specified)))    

# put the free (formerly unknown) displacements into the displacement vector u
# np.put(u,u_free,u_unknown)
for i in u_specified:
    u_unknown = u_unknown.row_insert(int(i), sp.Matrix([0]))
    
u = u_unknown    

In [44]:
u

Matrix([
[                   0],
[                   0],
[-19*L**3*P/(132*E*I)],
[  -7*L**2*P/(44*E*I)],
[                   0],
[  13*L**2*P/(44*E*I)]])

In [46]:
F = np.dot(np.array(K),np.array(u))

In [48]:
sp.Matrix(F)

Matrix([
[ 17*P/22],
[6*L*P/11],
[      -P],
[       0],
[  5*P/22],
[       0]])